In [1]:
import datacube
from datacube.testutils import io

dc = datacube.Datacube(app="L1_L2_convenience")

In [2]:
# convenience function to find Level 1 scenes for surface reflectance - only works with USGS collection 1 products!
def find_l1_paths_for_id(product, id):
    if product not in ['ls8_usgs_sr_scene','ls7_usgs_sr_scene','ls5_usgs_sr_scene']:
        raise ValueError('product {product} is not in: ls8_usgs_sr_scene, ls7_usgs_sr_scene, ls5_usgs_sr_scene'.format(product))
    
    if id is None:
        raise ValueError('id cannot be None')
    
    ds = dc.find_datasets(product=product,id= id)
    
    if not ds[0]:
        raise ValueError('id {id} not found'.format(id))
    
    # arbitrarily grab the S3 path for the nir band
    md = ds[0].metadata_doc['image']['bands']['nir']['path']
    
    # split file path into its constituents 
    collection, platform, path, row, year, month, day, file = md[md.find('c1'):].split('/')
    
    # string prefix is the landsat-pds bucket with s3 protocol
    prefix = 's3://landsat-pds'
    
    # file suffixes for the green and swir1 bands
    suffix_b3 = '_B3.TIF'
    suffix_b6 = '_B6.TIF'
    
    c1_id = file[0:file.rfind('_sr_band5.tif')]
    
    # reconstruct what we hope will be a valid S3 path
    # please note the capitilzation of the platform code and the 0 padding of the path and row
    green_path = '/'.join([prefix,collection,platform.capitalize(),path.zfill(3),row.zfill(3),c1_id,c1_id]) + suffix_b3
    swir1_path = '/'.join([prefix,collection,platform.capitalize(),path.zfill(3),row.zfill(3),c1_id,c1_id]) + suffix_b6
    
    return green_path, swir1_path

In [3]:
green_path, swir1_path = find_l1_paths_for_id('ls8_usgs_sr_scene','13b9c5f0-65f0-5d2e-8473-5bbfc92a9673')

In [4]:
l1_green = io.rio_slurp_xarray(green_path)
l1_swir1 =  io.rio_slurp_xarray(swir1_path)

RasterioIOError: Access Denied

In [ ]:
l1_green

In [ ]:
#12 Jan 2019
#green_path, swir1_path = find_l1_paths_for_id('ls8_usgs_sr_scene', '9eeccf9d-bd4c-5300-8e88-a8c72a7a2ef1')
                                              
                                              
# That cloudy day in jul2018
#'493e0037-3d82-5bcb-8ddf-f66ce3aea185'

In [ ]:
l1_green = io.rio_slurp_xarray(green_path)
l1_swir1 =  io.rio_slurp_xarray(swir1_path)

In [ ]:
l1_green

In [ ]:
l1_swir1

In [5]:
green_path

's3://landsat-pds/c1/L8/204/052/LC08_L1TP_204052_20180109_20180119_01_T1/LC08_L1TP_204052_20180109_20180119_01_T1_B3.TIF'

In [6]:
swir1_path

's3://landsat-pds/c1/L8/204/052/LC08_L1TP_204052_20180109_20180119_01_T1/LC08_L1TP_204052_20180109_20180119_01_T1_B6.TIF'